In [ ]:
import scrapy

from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.selector import Selector
from scrapy.utils.log import configure_logging

In [10]:
# class OlxSpider(scrapy.Spider):
#     name = "olx"
# 
#     def start_requests(self):
#         urls = [
#             'https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/3-camere/cluj-napoca/'
#         ]
#         for url in urls:
#             yield scrapy.Request(url=url, callback=self.parse)
# 
#     def parse(self, response):
#         print(response)
#         # page = response.url.split("/")[-2]
#         # filename = 'quotes-%s.html' % page
#         # with open(filename, 'wb') as f:
#         #     f.write(response.body)
#         # self.log('Saved file %s' % filename)

In [ ]:
class Olx(scrapy.Spider):
    name = "olx"
    start_urls = [
        'https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/3-camere/cluj-napoca/',
    ]
    
    def parse(self, response):
        
            

configure_logging()
runner = CrawlerRunner()
runner.crawl(Olx)
# runner.crawl(MySpider2)
d = runner.join()
d.addBoth(lambda _: reactor.stop())

reactor.run() # 

2020-04-02 17:35:43 [scrapy.crawler] INFO: Overridden settings:
{}


2020-04-02 17:35:43 [scrapy.extensions.telnet] INFO: Telnet Password: 12b4db51b912c6c3


2020-04-02 17:35:43 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']


2020-04-02 17:35:43 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']


2020-04-02 17:35:43 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']


2020-04-02 17:35:43 [scrapy.middleware] INFO: Enabled item pipelines:
[]


2020-04-02 17:35:43 [scrapy.core.engine] INFO: Spider opened


2020-04-02 17:35:43 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)


2020-04-02 17:35:43 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6029


ReactorNotRestartable: 